In [21]:
import pandas as pd
import numpy as np
import json
from scipy import sparse as sp
from tqdm.notebook import tqdm
from collections import defaultdict

In [22]:
import implicit

In [23]:
import sys
sys.path.append('../')

from src.utils import get_shard_path
from src.utils import ProductEncoder, make_coo_row
from src.metrics import normalized_average_precision

In [24]:
product_encoder = ProductEncoder('data/raw/products.csv')

In [25]:
valid_data = [json.loads(l) for l in open(get_shard_path(7))][:3000]

In [26]:
def make_coo_row(transaction_history, product_encoder: ProductEncoder):
    idx = []
    values = []

    items = []
    for trans in transaction_history:
        items.extend([i["product_id"] for i in trans["products"]])
    n_items = len(items)

    for pid in items:
        idx.append(product_encoder.toIdx(pid))
        values.append(1.0 / n_items)

    return sp.coo_matrix(
        (np.array(values).astype(np.float32), ([0] * len(idx), idx)), shape=(1, product_encoder.num_products),
    )

In [27]:
rows = []
for shard_id in range(4):
    for js in tqdm(json.loads(l) for l in open(get_shard_path(shard_id))):
        rows.append(make_coo_row(js["transaction_history"], product_encoder))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [28]:
X_sparse = sp.vstack(rows).tocsr()

In [29]:
X_sparse.shape

(183938, 43038)

# Cosine

In [62]:
model = implicit.nearest_neighbours.CosineRecommender(K=4)
model.fit(X_sparse)

/home/sap/src/personal/recommendation-systems/retailhero-recomender-baseline/.venv/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.05524158477783203 seconds
  warnings.warn(


  0%|          | 0/43038 [00:00<?, ?it/s]

In [63]:
m_ap = []
for js in tqdm(valid_data):
    row_sparse = make_coo_row(js["transaction_history"], product_encoder).tocsr()
    raw_recs = model.recommend(0, row_sparse, N=30, filter_already_liked_items=False, recalculate_user=True)
    recommended_items = product_encoder.toPid(raw_recs[0])
    gt_items = js["target"][0]["product_ids"]
    m_ap.append(normalized_average_precision(gt_items, recommended_items, k=30))
print(np.mean(m_ap))

  0%|          | 0/3000 [00:00<?, ?it/s]

0.1301746384436408


In [64]:
! mkdir -p ../tmp/i2i

In [65]:
import pickle
pickle.dump(model, open("../tmp/i2i/cosine.pkl", "wb"))